In [ ]:
!pip install pyspark==3.3.1

In [ ]:
import pandas as pd
import numpy as np

import scipy.sparse as sparse

import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, StringType

# ----------------------
import bisect
import glob

In [ ]:
# not ness
def age_bucket(x):
    return bisect.bisect_left([-1,25,35,45,55,65], x)
#test and train url
url_both = pd.read_csv('/kaggle/input/test-mts/train_test.csv')['0'].to_list()
#tar id
target_id = pd.read_parquet('/kaggle/input/test-mts/targets.pqt')['user_id'].to_list()

In [ ]:
spark = SparkSession.builder \
    .master('local') \
    .appName("MTS") \
    .getOrCreate()

In [ ]:
df = spark.read.parquet('/kaggle/input/test-mts/MTS/*.par*')
df.printSchema()

____________________
## URL part

In [ ]:
# pivot urls top per user
def top_urls(path: str, num: int = 11) -> None:
    urls = df.select('user_id','url_host','request_cnt')
    urls = urls.groupby('user_id', 'url_host').agg(F.sum('request_cnt').alias('rsum'))
    win = Window().partitionBy('user_id').orderBy(F.col('rsum').desc())
    urls = urls.withColumn('row_num',  F.row_number().over(win)).where(F.col('row_num') < (num + 1))
    urls.groupBy('user_id').pivot('row_num').agg(F.first('url_host')).write.csv(path, header='true')

In [ ]:
def url_more_one_req(path: str) -> None:
    win = Window().partitionBy('user_id').orderBy(F.col('sum(request_cnt)').desc())
    df.select('user_id','url_host','request_cnt').filter(F.col('request_cnt') > 1) \
                                                 .groupby('user_id','url_host').agg(F.sum('request_cnt')) \
                                                 .withColumn('row_num', F.row_number().over(win)) \
                                                 .write.csv(path, header='true')

In [ ]:
# social network part
def social_network(path: str) -> None:
    sn_list = ['m.zen.yandex.ru', 'dzen.ru', 'zen.yandex.ru', 
               'm.vk.com','vk.com',
               'm.ok.ru', 'm.odnoklassniki.ru', 'ok.ru',
               'm.facebook.com', 'facebook.com', 'l.facebook.com',
               'instagram.com', 'l.instagram.com', 
               't.co', 'twitter.com',
               'youtube.com',
               'linkedin.com']
    df = df.select('user_id', 'url_host', 'request_cnt')
    df = df.filter(F.col('url_host').isin(sn_list))
    df.groupby('user_id', 'url_host').sum('request_cnt').write.csv(path, header='true')

In [ ]:
def list_of_url(path: str) -> None:
    df = df.select('user_id', 'url_host', 'request_cnt')
    df = df.groupby('user_id', 'url_host').sum('request_cnt')
    #win = Window().partitionBy('user_id').orderBy(F.col('sum(request_cnt)').desc())
    #df = df.withColumn('row_num', F.row_number().over(win))
    #df.write.csv(path, header='true')
    df.groupBy('user_id').agg(F.concat_ws(' ', F.collect_set('url_host')).write.csv(path, header='true')

In [ ]:
# old. clear URLS
def url_prepare(df_temp):
    #all_url = pd.read_csv('/kaggle/input/test-mts/list_best_uld_2.csv')['0'].to_list()

    #df_temp.groupBy('user_id', 'url_host').agg(F.sum('request_cnt').alias('part_sum')) \
    #       .withColumn('all_sum', F.sum('part_sum').over(Window.partitionBy("user_id"))) \
    #       .write.csv('urlss', header='true')

    pattern = '(sun\d+\-.*?\.)'
    df_temp = df_temp.withColumn('url_host', F.when(F.col('url_host').like('%.turbopages%'), 
                                          F.regexp_replace('url_host', '--', '‰')).otherwise(F.col('url_host'))) \
                     .withColumn('url_host', F.when(F.col('url_host').like('%.turbopages%'), 
                                          F.regexp_replace('url_host', '-', '.')).otherwise(F.col('url_host'))) \
                     .withColumn('url_host', F.when(F.col('url_host').like('%.turbopages%'), 
                                          F.regexp_replace('url_host', '‰', '-')).otherwise(F.col('url_host'))) \
                     .withColumn('url_host',  F.regexp_replace('url_host', r'.turbopages', ''))                 \
                     .withColumn('url_host', F.regexp_replace('url_host', pattern, ''))
    # part1 mini-clear
    return df_temp


#    df_temp.groupBy('user_id').agg(F.concat_ws(" ", F.collect_set("url_host")).alias("urles")).write.csv('urles_v9', header='true')

#    pattern = '^(\w{1}\.)|(sun.*?\.)|(www\.)'
#    pattern = '(sun\d+\-.*?\.)'
#    pattern_2 = '\..\.'
#    df_temp = df_temp.withColumn('url_host', F.regexp_replace('url_host', pattern, ''))
#    df_temp = df_temp.withColumn('url_host', F.regexp_replace('url_host', pattern_2, '.'))

#     save url to text
#    df_temp.groupBy('user_id').agg(F.concat_ws(" ", F.collect_set("url_host")).alias("urles")).write.csv('urles_v0', header='true')
#    df_temp.groupBy('user_id').agg(F.concat_ws(" ", F.collect_list("url_host")).alias("urles")).write.csv('urles_v0list', header='true')

#     group_one = '.(\w*)$'
#     df_ur = df_temp.withColumn('loc_url', F.regexp_extract('url_host', group_one, 1))

#     df_temp = df_temp.withColumn('url_host', F.regexp_replace('url_host', '.\w*$', ''))

#     pattern_miniurl = '([\w_-]*\.?[\w_-]+$)'
#     df_temp = df_temp.withColumn('url_host', F.regexp_extract('url_host', pattern_miniurl, 1))


#     df_temp = df_temp.withColumn('url_host', F.when(F.col('url_host').like('%livejournal%'), 
#                                           F.lit('livejournal')).otherwise(F.col('url_host')))
    
    
#     df_temp.groupBy('user_id').agg(F.concat_ws(" ", F.collect_set("url_host")).alias("urles")).write.csv('urles_v1', header='true')
#     df_temp.groupBy('user_id').agg(F.concat_ws(" ", F.collect_list("url_host")).alias("urles")).write.csv('urles_v1list', header='true')

#     df_temp = df_temp.withColumn('url_host', F.when(~F.col('url_host').isin(all_url), \
#                                          F.lit('Other')).otherwise(F.col('url_host')))
#     # save url to text
   
#     df_temp.groupBy('user_id').agg(F.concat_ws(" ", F.collect_set("url_host")).alias("urles")).write.csv('urles_v2', header='true')
#     df_temp.groupBy('user_id').agg(F.concat_ws(" ", F.collect_list("url_host")).alias("urles")).write.csv('urles_v2list', header='true')

#     df_temp.groupBy('user_id', 'url_host').agg(F.sum('request_cnt').alias('sums_url')).write.csv('optuna', header='true')
#     df_temp.write.parquet('before_group.parquet')
    
#     df.groupBy('user_id').agg(F.count_distinct('url_host').alias('uniq_url'), \
#                           F.count('url_host').alias('all_url')) \
#                     .withColumn('url_diff', F.col('uniq_url') / F.col('all_url')).write.csv('url_count', header='true')
    
#     df.groupBy('user_id').agg(F.sum('request_cnt').alias('sums'), 
#                               F.count_distinct('date').alias('num_days'), 
#                               (F.datediff(F.max('date'), F.min('date')) + 1).alias('delta_date')).alias('d')

____________________________
## GEO part

In [ ]:
# 2 top region per user 
def top_reg(path: str):
    regions = df.select('user_id', 'region_name', 'request_cnt')
    win = Window().partitionBy('user_id').orderBy(F.col('rsum').desc())
    regions = regions.groupby('user_id', 'region_name').agg(F.sum('request_cnt').alias('rsum')) \
                     .withColumn('row_num',  F.row_number().over(win)).where(F.col('row_num') < 3)
    regions =  regions.groupBy('user_id').pivot('row_num').agg(F.first('region_name')).toPandas()
    regions.columns = ['user_id', 'first_reg','sec_reg']
    regions['sec_reg'] = df2['sec_reg'].fillna(df2['first_reg'])
    regions.to_parquet(f'{path}.pqt')

In [ ]:
# all unique region per user. text 
def region_list(path: str) -> None:
    df.select('user_id', 'region_name').groupBy('user_id')  \
                                       .agg(F.concat_ws(" ", F.collect_set("region_name")).alias("regions")) \
                                       .write.csv(path, header='true')

In [ ]:
# seq city per user, weekdays and weekends
# len of seq

# clear seq
def cit(data):
    city_list = data.split('_')
    res = [city_list[0]]
    for itm in city_list[1:]:
        if res[-1] != itm:
            res.append(itm)
    return '_'.join(res)

# create data
def mirgation_city(path: str, is_pivot: bool = True) -> None:
    df = df.select('user_id','date', 'city_name')
    win = Window().partitionBy('user_id').orderBy(F.col('date').desc())
    df = df.groupBy('user_id','date', 'city_name').count()
    df = df.withColumn('row_num',  F.row_number().over(win))
    df = df.withColumn('dayw', F.dayofweek('date'))
    df.select('user_id','city_name','row_num', 'dayw').write.csv(path, header='true')
    
    if is_pivot:
        filenames = glob.glob(f'/kaggle/working/{path}/*.csv')
        res = pd.concat(pd.read_csv(itm) for itm in filenames)
        
        res['dayw'] = res['dayw'].isin([1,7]).astype('int')
        res = res.pivot_table(index='user_id', 
                              columns='dayw', 
                              values='city_name', 
                              aggfunc=lambda x: '_'.join(x))
        res = res.reset_index()
        res.columns = ['user_id', 'weekend_list', 'weekdays_list']
        
        res['weekend_list'] = res['weekend_list'].map(cit)
        res['weekdays_list'] = res['weekdays_list'].map(cit)
        res['wend_cnt_city'] = res['weekend_list'].str.count('_')
        res['wdays_cnt_city'] = res['weekdays_list'].str.count('_')
        
        res['diff'] = res['wend_cnt_city'] - res['wdays_cnt_city'] 
        res.to_parquet(f'{path}.pqt')

__________________________
## Dates

In [ ]:
# uniq urls per user. weekend weekdays
def weeks(path: str) -> None:
    # 5 - четверг
    res = df.select('user_id', 'date', 'url_host').withColumn('date', F.dayofweek('date')) 
    res = res.withColumn('date', F.when(F.col('date').isin([1,7]), 0).otherwise(1))
    res = res.groupBy('user_id').pivot('date').agg(F.concat_ws(" ", F.collect_set("url_host")).alias("urles")).toPandas()
    res.columns = ['user_id', 'wend', 'wdays']
    res['wend'] = res.apply(lambda x: ' '.join(set(x['wend'].split(' ')) - set(x['wdays'].split(' '))), axis=1)
    res['wdays'] = res.apply(lambda x: ' '.join(set(x['wdays'].split(' ')) - set(x['wend'].split(' '))), axis=1)
    res.to_parquet(f'{path}.pqt')

In [ ]:
# URLs->word per user. part of day
def word_days(path: str, part_day: str, is_explode: bool = True) -> None:
    # urls
    # evening, day, morning,  night
    if part_day:
        urls = df.filter(F.col('part_of_day') == part_day).select('user_id', 'url_host', 'request_cnt')
    else:
        urls = df.select('user_id', 'url_host', 'request_cnt')
   
    urls = urls.groupby('user_id', 'url_host').sum('request_cnt')

    pattern = '\.com\.|\.(\w*)$'
    urls = urls.withColumn('clear_url', F.regexp_replace('url_host', pattern, ''))
    urls = urls.withColumn('clear_url', F.regexp_replace('clear_url', '[^A-Za-z]', ' '))
    urls = urls.withColumn('clear_url', F.regexp_replace('clear_url', '( |^).( |$)', ' '))
    urls = urls.withColumn('clear_url', F.regexp_replace('clear_url', ' +', ' '))

    if is_explode:
        urls = urls.withColumn('clear_url', F.explode(F.split('clear_url',' ')))
        urls = urls.filter(F.col('clear_url') != '')
        urls = urls.groupby('user_id', 'clear_url').agg(F.sum('rsum').alias('rsum'))
        
    
    win = Window().partitionBy('user_id').orderBy(F.col('rsum').asc())
    urls = urls.withColumn('row_num', F.row_number().over(win))
    
    urls.write.csv(path, header='true')

    # data "leak"
    #urls = urls.join(target, on='user_id', how='inner')
    #urls = urls.select('clear_url','age','rsum')
    #urls.groupBy('clear_url').pivot('age').agg(F.sum('rsum'), F.count('rsum')).write.csv('age', header='true')


In [ ]:
def last_days_count():
    df = df.select('user_id', 'date').groupby('user_id', 'date').count()
    win = Window().partitionBy('user_id').orderBy(F.col('date').desc())
    df = df.withColumn('row_num', F.row_number().over(win)).where(F.col('row_num') < 6)
    df.groupby('user_id').pivot('row_num').agg(F.sum('count')).write.csv('last5day', header='true')

__________________________________
## Other

In [ ]:
# cpe_manufacturer_name. old
# data cont only 1 phone for user
def phones(path: str) -> None:
    popular = ['Apple', 'Samsung', 'Huawei', 'Xiaomi']
    df_temp = df.select('user_id','cpe_manufacturer_name')
    test = df_temp.withColumn('cpe_manufacturer_name', F.when(~df_temp.cpe_manufacturer_name.isin(popular), 
                                                              F.lit('Other')).otherwise(df_temp.cpe_manufacturer_name))
    result = test.dropDuplicates(['user_id']).alias('phones')
    result.write.csv(path, header='true')

In [ ]:
# mean price for phone. old
# by cpe_manufacturer_name
def mean_price(path: str) -> None:
    popular = ['Apple', 'Samsung', 'Huawei', 'Xiaomi']
    df_temp = df.select('price','cpe_manufacturer_name','user_id')
    df_temp = df_temp.withColumn('cpe_manufacturer_name', F.when(~df_temp.cpe_manufacturer_name.isin(popular), 
                                                                 F.lit('Other')).otherwise(df_temp.cpe_manufacturer_name))
    win = Window.partitionBy('cpe_manufacturer_name')
    res = df_temp.withColumn('new_price', F.avg(df_temp.price).over(win))
    res = res.withColumn('price', F.when(F.isnull(res.price), res.new_price).otherwise(res.price))
    result = res.groupby('user_id').agg(F.mean('price').alias('price'))
    result.write.csv(path, header='true')

In [ ]:
# OS type. old
def ostypes(path: str) -> None:
    df_temp = df.select('cpe_model_os_type', 'user_id')
    df_temp = df_temp.withColumn('cpe_model_os_type', F.regexp_replace('cpe_model_os_type', 'Apple iOS', 'iOS'))
    result = df_temp.groupBy('user_id').agg(F.first('cpe_model_os_type'))
    result.write.csv(path, header='true')

_______________________________
## For URL Cluster

In [ ]:
def modas(col):
    win  = Window().partitionBy('url_host').orderBy(F.col('sum(request_cnt)').desc())
    df_moda = df.select(col, 'url_host', 'request_cnt').groupby('url_host', col) \
                                .agg(F.sum('request_cnt'))
    df_moda = df_moda.withColumn('row_num', F.row_number().over(win)).where(F.col('row_num') == 1)
   # df_moda.write.csv(f'{col}_moda', header='true')
    df_moda = df_moda.select('url_host', col)
    return df_moda

def modas2():
    reg = modas('region_name')
    cpe = modas('cpe_manufacturer_name')
    model = modas('cpe_model_name')
    oss = modas('cpe_model_os_type')
    day_part = modas('part_of_day')
    # group URLs
    another = df.groupBy('url_host').agg(F.mean('price').alias('price'), 
                           F.count_distinct('user_id').alias('users'), 
                           F.datediff(F.current_date(), F.max('date')).alias('last_date'), 
                           F.mean('request_cnt').alias('mean_req'),
                           F.sum('request_cnt').alias('sum_req'))
    urls = another.join(reg, on='url_host', how='left')
    urls = urls.join(cpe, on='url_host', how='left')
    urls = urls.join(model, on='url_host', how='left')
    urls = urls.join(oss, on='url_host', how='left')
    urls = urls.join(day_part, on='url_host', how='left')
    urls.write.csv('urls', header='true')

____________________________
## Data "Leak"

In [ ]:
# IDs
def male_and_female(path: str) -> list:
    targets = pd.read_csv(path)
    males = targets[targets['is_male'] == 1]['user_id'].to_list()
    females = targets[targets['is_male'] == 0]['user_id'].to_list()
    return [males, females]

def user_cat(num):
    targets = pd.read_parquet('/kaggle/input/test-mts/targets.pqt')
    targets['age'] = targets['age'].map(age_bucket)
    nums = targets[targets['age']  == num]['user_id'].to_list()
    return nums

In [ ]:
def ages_perc(path: str, age_: int) -> None:
    df = df.select('user_id', 'url_host', 'request_cnt').filter(F.col('url_host').isin(url_both))
    df_one = df.filter(F.col('user_id').isin(user_cat(age_))).groupBy('url_host') \
                                                             .agg(F.sum('request_cnt').alias('ft_sum'), \
                                                                  F.count('request_cnt').alias('ft_count'))
    df_one.write.csv(path, header='true')

In [ ]:
def f_m(df):
    df = df.select('user_id', 'url_host', 'request_cnt')
    males, females = male_and_female('/kaggle/input/test-mts/is_male_target_414724.csv')
    df_males = df.filter(F.col('user_id').isin(males)).groupBy('url_host') \
                                                      .agg(F.sum('request_cnt').alias('male_sum'),\
                                                           F.count('request_cnt').alias('male_count'))
    df_males.write.csv('males', header='true')
    df_females = df.filter(F.col('user_id').isin(females)).groupBy('url_host') \
                                                          .agg(F.sum('request_cnt').alias('female_sum'),\
                                                               F.count('request_cnt').alias('female_count'))
    df_females.write.csv('females', header='true')
    df_both = df.groupBy('url_host') \
                .agg(F.sum('request_cnt').alias('both_sum'),\
                     F.count('request_cnt').alias('both_count'))
    df_both.write.csv('both', header='true')